In [59]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scipy.stats import stats
from utils.average_spectra import average_dataframe
from utils.gray2ndvi import gray2ndvi, threshold
sns.set(style="whitegrid", color_codes=True)
plt.rcParams['figure.figsize'] = (12, 8)

In [60]:
MIN_NDVI = 0.178432
MAX_NDVI = 0.368569

In [61]:
rows, cols = 8, 3
path = "../data/UAV/orange1"

# 160x160 is the image dimension
endvi = np.zeros((rows, cols, 160, 160))

for i in range(rows):
    for j in range(cols):
        filename = path + "/{}{}.png".format(j+1, chr(i+65))
        try:
            img = cv2.imread(filename)
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray_img = cv2.resize(gray_img, (160, 160))
            endvi[i, j] = gray2ndvi(gray_img, MIN_NDVI, MAX_NDVI)
            endvi[i, j] = threshold(endvi[i, j], MIN_NDVI, MAX_NDVI)
        except:
            endvi[i, j] = np.nan

In [62]:
endvi_mean = np.zeros((rows, cols))

for i in range(rows):
    for j in range(cols):
        endvi_mean[i, j] = np.nanmean(endvi[i, j])

In [63]:
endvi_mean[0, 0]

0.33490705474047566

In [64]:
endvi_mean = endvi_mean.flatten("F")
endvi_mean = endvi_mean[~np.isnan(endvi_mean)]
endvi_mean

array([ 0.33490705,  0.32639327,  0.32530164,  0.33234782,  0.32851721,
        0.31904888,  0.30175293,  0.30014589,  0.33799363,  0.32851576,
        0.33660884,  0.34271504,  0.34299348,  0.3243784 ,  0.29834162,
        0.29434938,  0.34093913,  0.33380244,  0.33162118,  0.33576664,
        0.3298913 ,  0.32565457,  0.29850008,  0.29794325])

In [65]:
spectra_12 = pd.read_csv("../data/Unprocessed/2017-05-12.csv")
spectra_19 = pd.read_csv("../data/Unprocessed/2017-05-19.csv")

In [89]:
NIR = (750, 1075)
BLUE = (450, 515)
GREEN = (520, 600)
wavelength_index = {k:v for k, v in zip(spectra_12["Wavelength"], range(751))}

def preprocess_spectra(df):
    return average_dataframe(df, 20).drop("Wavelengths", 1).as_matrix()

def ENDVI(v):
    nir = v[wavelength_index[NIR[0]]:wavelength_index[NIR[1]]].mean(axis=0)
    green = v[wavelength_index[GREEN[0]]:wavelength_index[GREEN[1]]].mean(axis=0)
    blue = v[wavelength_index[BLUE[0]]:wavelength_index[BLUE[1]]].mean(axis=0)
    return ((nir + green) - (2 * blue)) / ((nir + green) + (2 * blue))

In [90]:
spectra = np.zeros((4, 8, 751))
spectra[0:2] = preprocess_spectra(spectra_12).T.reshape((2,8,751), order="C")
spectra[2:4] = preprocess_spectra(spectra_19).T.reshape((2,8,751), order="C")

In [91]:
spectra = spectra[:3]
spectra = spectra.reshape((24, 751))
spectra.shape

(24, 751)

In [97]:
endvi = [ENDVI(spectra[i]) for i in range(spectra.shape[0])]

In [98]:
stats.pearsonr(endvi_mean, endvi)

(-0.0030744051412641997, 0.98862463482722185)

In [100]:
endvi_mean.mean(), np.array(endvi).mean()

(0.32368456030928522, 0.7950977283478956)